# Questions
- What does it mean when OSRM matching route's `matchings` array has length greater than 1?
  - From [Explanation of OSRM Map-Matching Algorithm and API Response](https://github.com/Project-OSRM/osrm-backend/issues/2933#issuecomment-249068441) 
  > The elements of matchings are not connected - if your trace matches without
  > error, then matchings array should only have 1 entry. The fact that you've got 4
  > means that OSRM split your coordinates into 4 partial matches with gaps between.
  > The matching algorithm is probabilistic and your GPS data is noisy - it's not
  > going to give you a perfect result every time.

In [1]:
import sys
import os
import itertools
import json
from urllib.error import URLError, HTTPError
from string import Template

from typing import TypedDict, List, Annotated

import pprint

import pandas as pd
import geopandas as gpd

import networkx as nx
import osmnx as ox

In [2]:
local_modules_path = os.path.abspath('.')
print(local_modules_path)

if local_modules_path not in sys.path:
    print('append local_modules_path')
    sys.path.append(local_modules_path)

from src.osm.osm import OSM
from src.ris import RISMilepoint
from src.common import BrokenInvariantError

/home/avail/AVAIL/avail-transportation-resiliency-metrics/processing/osm_nysdot_ris_conflation
append local_modules_path


In [14]:
from src.osm.osrm import (
    OSRM,
    get_tracepoints_linestrings,
    extract_osm_node_chain,
    get_tracepoints_multipolygon,
    get_tracepoints_distances_summary,
    assert_matching_legs_coorespond_to_feature_coords_pairs,
    assert_matching_legs_coorespond_to_nonnone_tracings_length,
    assert_osrm_matching_node_chain_is_a_connected_path,
    assert_osrm_matching_node_chain_is_a_connectable_path
)

In [15]:
##### Imports Config #####
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
## CONFIG ##
osm_subnet_name = 'county-36001_new-york-240101'
ris_subnet_name = 'county-36001_nysdot_ris'

osrm_host = 'http://127.0.0.1:5000'

osm_pbf_path = f'./initial_data/{osm_subnet_name}.osm.pbf'
ris_gis_fpath = f'./create_nysdot_ris_county_subnets/{ris_subnet_name}.gpkg'

In [17]:
osrm = OSRM(host=osrm_host, osm_subnet_name=osm_subnet_name)

In [102]:
ris = RISMilepoint(ris_gis_fpath)
ris_gdf = ris.gdf

/home/avail/AVAIL/avail-transportation-resiliency-metrics/.direnv/python-3.10.12/lib/python3.10/site-packages/pyogrio/raw.py:196: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(


In [19]:
len(ris_gdf)

5353

In [147]:
# for d in list(gdf.iterfeatures()):
matches_stats = [0, 0, 0]

ris_feature_osm_matchings = dict()

for feature in list(ris_gdf.iterfeatures()):
    # pprint.pprint(feature)
    ris_route_id = feature['id']

    fwd_ways_chain = None
    try:
        match_response = osrm.match(feature, reverse=False)
        
        fwd_nodes_chain = extract_osm_node_chain(match_response['matching_response'])

        # try:
        #     fwd_ways_chain = osm_node_chain_to_ways_chain2(fwd_nodes_chain)
        # except:
        #     # print(ris_route_id, 'no fwd way chain')
        #     pass
        
    except HTTPError as e:
        # resp_json = json.loads(e.fp.readlines()[0])
        # print(resp_json['message'])
        pass
    except BrokenInvariantError as ex:
        # print(ex)
        pass

    bwd_ways_chain = None
    try:
        match_response = osrm.match(feature, reverse=True)
        
        bwd_nodes_chain = extract_osm_node_chain(match_response['matching_response'])

        # try:
        #     bwd_ways_chain = osm_node_chain_to_ways_chain2(bwd_nodes_chain)
        # except:
        #     # print(ris_route_id, 'no bwd way chain')
        #     pass
        
    except HTTPError as e:
        # resp_json = json.loads(e.fp.readlines()[0])
        # print(resp_json['message'])
        pass
    except BrokenInvariantError as ex:
        # print(ex)
        pass

    matches_stats[bool(fwd_nodes_chain) + bool(bwd_nodes_chain)] += 1

    ris_feature_osm_matchings[ris_route_id] = {
        'fwd_nodes_chain': fwd_ways_chain,
        'bwd_nodes_chain': bwd_ways_chain
    }

print(matches_stats)
# pprint.pprint(ris_feature_osm_matchings)

[0, 0, 5353]


In [21]:
pd.DataFrame(matches_stats, ['no matches', 'one direction matched', 'both directions matched'], ['count'])

,count
no matches,0
one direction matched,0
both directions matched,5353


# NEXT
1. Which and Why RIS Routes get
    1. No matches in either direction?
    2. No matches in one direction?
        1. Of these, determine if the roadway is "oneway"
            1. Is the segment in the Ev_RIS_One_Way Table?
            2. Are the other direction's matched OSM Ways oneway?
2. Which edges in NetworkX do not get any RIS Matches?

In [179]:
ris_oneway_gdf = gpd.read_file(
    ris_gis_fpath,
    rows=slice(00, 10_000), ## FIXME: Dev subset
    layer='Ev_RIS_One_Way',
    columns=['ROUTE_ID', 'FROM_MEASURE', 'TO_MEASURE'],
)

ris_oneway_gdf = ris_oneway_gdf.reindex(columns=['ROUTE_ID', 'FROM_MEASURE', 'TO_MEASURE'])

In [192]:
ris_oneway_routeids = set(ris_oneway_gdf['ROUTE_ID'])

In [194]:
len(ris_oneway_routeids)

557

In [206]:
match_oneway_ris_routeids = set([k for k,v in ris_feature_osm_matchings.items() if (bool(v['fwd_ways_chain']) + bool(v['bwd_ways_chain'])) == 1])

In [207]:
len(match_oneway_ris_routeids)

169

In [209]:
intxn = ris_oneway_routeids.intersection(match_oneway_ris_routeids)
len(intxn)

99

In [213]:
ris_diff_match = ris_oneway_routeids.difference(match_oneway_ris_routeids)
match_diff_ris = match_oneway_ris_routeids.difference(ris_oneway_routeids)

print(len(ris_diff_match))
print(len(match_diff_ris))

458
70


In [67]:
feature = next(ris_gdf.iterfeatures())
fwd_osrm_match_response = osrm.match(feature, reverse=False)

fwd_osrm_match_response

{'osm_subnet_name': 'county-36001_new-york-240101',
 'matched_coordinates_sequence': ((-73.83292597957319, 42.62219003210422),
  (-73.83526994654903, 42.62277700425111),
  (-73.83675597479998, 42.623128011041075),
  (-73.83700705122955, 42.623187024112816),
  (-73.83761298405929, 42.623317990820965),
  (-73.83775799673563, 42.62334296752228),
  (-73.83813599087404, 42.623410019887274),
  (-73.8390110332388, 42.6235159713233),
  (-73.83935795500268, 42.62354902496127),
  (-73.84144600698293, 42.62373199123098),
  (-73.84186104798283, 42.62374798531408),
  (-73.84209603018489, 42.62373496130252),
  (-73.8423719989648, 42.623708033119314),
  (-73.8437470105856, 42.62351803408984),
  (-73.84391196632386, 42.623499977756246),
  (-73.84412303519447, 42.62348103385885),
  (-73.84436600944612, 42.623470967597534),
  (-73.8450179744612, 42.623480964981646),
  (-73.84528598101961, 42.623502038121195),
  (-73.84596798935581, 42.62360202801436),
  (-73.84718602530143, 42.623823040454795),
  (-73.8

In [69]:
nodes_chain = osrm.extract_osm_node_chain(fwd_osrm_match_response['matching_response'])


In [27]:
osm = OSM(osm_pbf_path)

In [28]:
osm.init_nx()

In [92]:
for u,v in zip(nodes_chain[:-1], nodes_chain[1:]):
    e = osm.nx.G.get_edge_data(u,v)
    if not e:
        print('no', u, v)

In [112]:
gdf = gpd.GeoDataFrame(
    { 'geometry': linestrings },
    geometry='geometry'
)

In [113]:
gdf.to_file(
    'tracepoints.gpkg',
    driver='GPKG',
    layer='tracepoints'
)

/home/avail/AVAIL/avail-transportation-resiliency-metrics/.direnv/python-3.10.12/lib/python3.10/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [78]:
# polys = get_tracepoint_multipolygon(fwd_osrm_match_response)
matchings_by_route_id = {}

for feature in list(ris_gdf.iterfeatures())[:]:
    id = feature['id']

    matchings_by_route_id[id] = { 'fwd': None, 'bwd': None}

    try:
        matchings_by_route_id[id]['fwd'] = osrm.match(feature, reverse=False)
    except:
        pass

    try:
        matchings_by_route_id[id]['bwd'] = osrm.match(feature, reverse=True)
    except:
        pass

In [ ]:
# polys = get_tracepoint_multipolygon(fwd_osrm_match_response)
matchings = {
    'id': [],
    
    'fwd_polys': [],
    'fwd_summary_stats': [],

    'bwd_polys': [],
    'bwd_summary_stats': []
}

for feature in list(ris_gdf.iterfeatures())[:]:
    matchings['id'].append(feature['id'])

    try:
        match_result = osrm.match(feature, reverse=False)
        matchings['fwd_polys'].append(get_tracepoints_multipolygon(match_result))
        matchings['fwd_summary_stats'].append(get_tracepoints_distances_summary(match_result))
    except:
        matchings['fwd_polys'].append(None)
        matchings['fwd_summary_stats'].append(None)

    try:
        match_result = osrm.match(feature, reverse=True)
        matchings['bwd_polys'].append(get_tracepoints_multipolygon(match_result))
        matchings['bwd_summary_stats'].append(get_tracepoints_distances_summary(match_result))
    except:
        matchings['bwd_polys'].append(None)
        matchings['bwd_summary_stats'].append(None)

In [151]:
fwd_gdf = gpd.GeoDataFrame(
    { 'id': matchings['id'], 'geometry': matchings['fwd_polys'] },
    geometry='geometry'
)

bwd_gdf = gpd.GeoDataFrame(
    { 'id': matchings['id'], 'geometry': matchings['bwd_polys'] },
    geometry='geometry'
)

In [182]:
fwd = [{ 'id': id, 'dir': 'fwd', **d } for id, d in zip(matchings['id'], matchings['fwd_summary_stats']) if d is not None]
bwd = [{ 'id': id, 'dir': 'bwd', **d } for id, d in zip(matchings['id'], matchings['bwd_summary_stats']) if d is not None]

summary_stats = gpd.GeoDataFrame(
    fwd + bwd,
    geometry=None
)

In [183]:
fwd_gdf.to_file(
    'matching_qa.gpkg',
    driver='GPKG',
    layer='fwd_match_polys',
)

bwd_gdf.to_file(
    'matching_qa.gpkg',
    driver='GPKG',
    layer='bwd_match_polys',
)

summary_stats.to_file(
    'matching_qa.gpkg',
    driver='GPKG',
    layer='summary_stats',
)

/home/avail/AVAIL/avail-transportation-resiliency-metrics/.direnv/python-3.10.12/lib/python3.10/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
/home/avail/AVAIL/avail-transportation-resiliency-metrics/.direnv/python-3.10.12/lib/python3.10/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [179]:
fwd = [{ 'id': id, 'dir': 'fwd', **d } for id, d in zip(matchings['id'], matchings['fwd_summary_stats']) if d is not None]
bwd = [{ 'id': id, 'dir': 'bwd', **d } for id, d in zip(matchings['id'], matchings['bwd_summary_stats']) if d is not None]

summary_stats = pd.DataFrame(fwd + bwd)
summary_stats.set_index(keys=['id', 'dir'], drop=True, inplace=True, verify_integrity=True)

summary_stats.sort_index(ascending=[True, False])

count      mean       std       min       25%       50%  \
id        dir                                                            
100074011 fwd   73.0  0.918661  0.615127  0.111085  0.467692  0.849903   
          bwd   73.0  6.964201  6.035555  0.111085  0.782613  7.637024   
100074012 fwd   61.0  5.611513  5.340648  0.111085  0.742910  1.452168   
          bwd   61.0  0.797721  0.580157  0.000000  0.396278  0.736046   
100077011 fwd  618.0  0.884485  0.768711  0.000000  0.270551  0.671549   
...              ...       ...       ...       ...       ...       ...   
318731011 bwd   14.0  0.670239  0.603828  0.000000  0.203824  0.398968   
318734011 fwd   31.0  1.039810  0.673818  0.138131  0.422609  1.017861   
          bwd   31.0  1.039810  0.673818  0.138131  0.422609  1.017861   
320627011 fwd   18.0  1.455460  1.258626  0.000000  0.568448  1.235760   
          bwd   18.0  1.455460  1.258626  0.000000  0.568448  1.235760   

                     75%        max  
id        dir                        
100074011 fwd   1.152951   3.368716  
          bwd  13.097125  13.689517  
100074012 fwd  11.365634  13.112608  
          bwd   1.013142   3.368716  
100077011 fwd   1.293616   5.021384  
...                  ...        ...  
318731011 bwd   1.166536   1.773773  
318734011 fwd   1.604329   2.501487  
          bwd   1.604329   2.501487  
320627011 fwd   1.990157   4.288904  
          bwd   1.990157   4.288904  

[10032 rows x 8 columns]

In [185]:
feature = next(ris_gdf.iterfeatures())
match_response = osrm.match(feature, reverse=False)

assert_matching_legs_coorespond_to_feature_coords_pairs(match_response)


In [15]:
stats = {
    'pass': 0,
    'fail': 0
}
for id, d in matchings_by_route_id.items():
    try:
        if d['fwd']:
            assert_matching_legs_coorespond_to_feature_coords_pairs(d['fwd'])
            stats['pass'] += 1
    except BrokenInvariantError as e:
        print(id, e)
        stats['fail'] += 1
    
    # assert_matching_legs_coorespond_to_feature_coords_pairs(d['bwd'])
    
stats

100081021 len(f_coords) = 376; len(legs) = 374
100479012 len(matchings) = 31
100495072 len(matchings) = 14
100500012 len(f_coords) = 19; len(legs) = 10
100513051 len(f_coords) = 396; len(legs) = 393
100514081 len(matchings) = 8
100515142 len(matchings) = 17
100564011 len(f_coords) = 132; len(legs) = 127
100588012 len(matchings) = 2
101875011 len(f_coords) = 7; len(legs) = 4
101954011 len(f_coords) = 4; len(legs) = 1
102425011 len(matchings) = 2
102493011 len(f_coords) = 25; len(legs) = 22
103542011 len(f_coords) = 47; len(legs) = 43
104043011 len(f_coords) = 13; len(legs) = 11
104303011 len(f_coords) = 10; len(legs) = 7
104588011 len(f_coords) = 9; len(legs) = 2
272537011 len(matchings) = 2
272720011 len(f_coords) = 25; len(legs) = 22
278973011 len(matchings) = 3
301491011 len(f_coords) = 14; len(legs) = 6
308445011 len(f_coords) = 40; len(legs) = 37
100147202 len(f_coords) = 351; len(legs) = 336
100290042 len(matchings) = 4
100348012 len(matchings) = 3
100480012 len(matchings) = 2
100

{'pass': 5094, 'fail': 214}

In [31]:
stats = {
    'pass': 0,
    'fail': 0
}
for id, d in matchings_by_route_id.items():
    try:
        if d['fwd']:
            assert_matching_legs_coorespond_to_nonnone_tracings_length(d['fwd'])
        if d['bwd']:
            assert_matching_legs_coorespond_to_nonnone_tracings_length(d['bwd'])
        stats['pass'] += 1
    except BrokenInvariantError as e:
        print(id, e)
        stats['fail'] += 1
    
stats

{'pass': 5353, 'fail': 0}

In [26]:
matchings_by_route_id['102536011']['fwd']

{'osm_subnet_name': 'county-36001_new-york-240101',
 'matched_coordinates_sequence': ((-73.8045670458203, 42.764316957811836),
  (-73.80459296085408, 42.764215018125824),
  (-73.80464098274666, 42.763973997112814),
  (-73.80462090520439, 42.76365209814051)),
 'matching_response': {'code': 'Ok',
  'matchings': [{'confidence': 0.981004235,
    'geometry': '_l_dGt}maMNBB?',
    'legs': [{'steps': [],
      'summary': '',
      'weight': 2.7,
      'duration': 2.7,
      'annotation': {'metadata': {'datasource_names': ['lua profile']},
       'datasources': [0, 0],
       'weight': [2.3, 0.4],
       'nodes': [41380118, 1520517005, 7369731936],
       'distance': [9.521970367, 2.014606306],
       'duration': [2.3, 0.4],
       'speed': [4.1, 5]},
      'distance': 11.5}],
    'weight_name': 'routability',
    'weight': 2.7,
    'duration': 2.7,
    'distance': 11.5},
   {'confidence': 0.979694056,
    'geometry': 'yi_dG~}maMH?r@C@?',
    'legs': [{'steps': [],
      'summary': '',
      '

In [52]:
c = osm.nx.G.copy()

# WARNING: Reindexing and exporting to GPKG throws the following error:
#            ValueError: cannot insert osmid, already exists
#
# The following workaround is suggested by the library's author:
#   https://github.com/gboeing/osmnx/issues/638#issuecomment-756948363
for node, data in c.nodes(data=True):
    if 'osmid' in data:
        data['osmid_original'] = data.pop('osmid')

osm_nx_nodes_gdf, osm_nx_edges_gdf = ox.graph_to_gdfs(c)

osm_nx_nodes_gdf.to_file(
    'osm_nx.gpkg',
    driver='GPKG',
    layer='nodes',
)

osm_nx_edges_gdf.to_file(
    'osm_nx.gpkg',
    driver='GPKG',
    layer='edges',
)


In [59]:
matchings_by_route_id['103454011']['fwd']

{'osm_subnet_name': 'county-36001_new-york-240101',
 'matched_coordinates_sequence': ((-73.90791004321686, 42.59424101447981),
  (-73.90780699275201, 42.594006967405946),
  (-73.90748101882093, 42.593054039523516),
  (-73.9073690068937, 42.592781993009915),
  (-73.9072239533102, 42.59239904252688),
  (-73.90724696169622, 42.592295967832),
  (-73.90728697684118, 42.592237993253825),
  (-73.90737194941404, 42.59218098777288),
  (-73.90754302126797, 42.59214200396111),
  (-73.90771301424678, 42.59209499462827),
  (-73.90781396818517, 42.59205804354852),
  (-73.90792303943059, 42.59198199542192),
  (-73.90796603186517, 42.591920987080464),
  (-73.90798498197441, 42.59179896179444),
  (-73.90774194150609, 42.590606015149994),
  (-73.90760101273747, 42.58998004327036),
  (-73.90758804808394, 42.58987797628865),
  (-73.90759522791268, 42.58980316320721)),
 'matching_response': {'code': 'Ok',
  'matchings': [{'confidence': 0.752362631,
    'geometry': '}d~bGpcbbM^MLCpBg@jA[n@QDCt@UTCBA',
    '

In [83]:
stats = {
    'pass': 0,
    'fail': 0
}

for id, d in matchings_by_route_id.items():
    ris_feature = ris_gdf.loc[id]

    try:
        if d['fwd']:
            if ris_feature['DIRECTION'] != '2':
                assert_osrm_matching_node_chain_is_a_connectable_path(osm.nx.G, d['fwd'])
        if d['bwd']:
            if ris_feature['DIRECTION'] != '1':
                assert_osrm_matching_node_chain_is_a_connectable_path(osm.nx.G, d['bwd'])
        stats['pass'] += 1
    except BrokenInvariantError as e:
        print(e)
        stats['fail'] += 1
    
stats

NO PATH: id=100147201, reversed=True, u=1362215178, v=60655289
id=100290041, reversed=True, u=10956463721, v=41524773, chain gap length=122
id=100290041, reversed=True, u=41524950, v=41306926, chain gap length=79
id=100290041, reversed=True, u=9173569525, v=41524849, chain gap length=32
id=100290041, reversed=True, u=41271065, v=41420607, chain gap length=52
id=100290041, reversed=True, u=41449071, v=10655540254, chain gap length=3
id=100348011, reversed=True, u=41475591, v=2545675933, chain gap length=116
id=100348011, reversed=True, u=2545675984, v=41288464, chain gap length=57
id=100348011, reversed=True, u=41251848, v=41263885, chain gap length=103
id=100348011, reversed=True, u=41475511, v=544355406, chain gap length=62
id=100348011, reversed=True, u=41258043, v=2545676011, chain gap length=121
id=100492011, reversed=True, u=41270160, v=41257167, chain gap length=154
id=100514081, reversed=False, u=41411743, v=2565929760, chain gap length=115
id=100514081, reversed=False, u=412595

{'pass': 5337, 'fail': 16}

In [84]:
ris_gdf.loc['100290041']

DIRECTION                                                       0
ROADWAY_TYPE                                                    3
ROUTE_NUMBER                                                   32
geometry        MULTILINESTRING ((-74.01763996151082 42.436055...
Name: 100290041, dtype: object

### DIRECTION Codes

-  0 - Primary Direction with Undivided Inventory
-  1 - Primary Direction with Divided Inventory
-  2 - Reverse Direction with Divided Inventory
-  3 - Reverse Direction with No Inventory

From [MEXIS_APP.BC_CONSULTING_NONAE_ADMIN](https://www.dot.ny.gov/portal/pls/portal/MEXIS_APP.BC_CONSULTING_NONAE_ADMIN.VIEWFILE?p_file_id=43687)

> Direction 2 and 3 polylines are included to help determine where the reverse
> direction deviates enough from the primary direction to warrant additional
> collection in the reverse direction.

In [151]:
"""We would expect to reject poor matches in one direction for following RIS Route Features."""

pd.set_option('display.max_rows', None)

ris_d = ris_gdf.copy()

ris_d['ROUTE_ID'] = ris_d.index

d = pd.merge(left=ris_d, right=ris_d, how='inner', on=['DOT_ID', 'COUNTY_ORDER']).sort_values(by=['DOT_ID', 'COUNTY_ORDER'])
d2 = d[d['DIRECTION_x'] < d['DIRECTION_y']][['ROUTE_ID_x', 'ROUTE_ID_y', 'DOT_ID', 'DIRECTION_x', 'DIRECTION_y', 'COUNTY_ORDER']]

# display(d2)

# Direction Codes
expect_only_fwd_route_ids = set(d2[d2['DIRECTION_x'] <= '1']['ROUTE_ID_x'])
expect_only_bwd_route_ids = set(d2[d2['DIRECTION_y'] > '1']['ROUTE_ID_y'])

In [174]:
"""
NOTICE: There are no longer any chain gaps in the output below.
TODO: Figure out why we get NO PATH errors. It would seem there'd alway be a path in the road network.
"""

stats = {
    'pass': 0,
    'fail': 0
}

for id, d in matchings_by_route_id.items():
    ris_feature = ris_gdf.loc[id]
    ris_direction = ris_feature['DIRECTION']
    try:
        if id not in expect_only_bwd_route_ids:
            if d['fwd']:
                assert_osrm_matching_node_chain_is_a_connectable_path(osm.nx.G, d['fwd'])
            else:
                raise Exception('NO MATCH')
            stats['pass'] += 1
    except BrokenInvariantError as e:
        print(f"ROUTE_ID={id} (FWD), DIRECTION={ris_direction};  ", e)
        stats['fail'] += 1
    except BaseException as e:
        print(f"ROUTE_ID={id} (FWD), DIRECTION={ris_direction};  ", e)
        stats['fail'] += 1

    try:
        if id not in expect_only_fwd_route_ids:
            if d['bwd']:
                assert_osrm_matching_node_chain_is_a_connectable_path(osm.nx.G, d['bwd'])
            else:
                raise Exception('NO MATCH')

            stats['pass'] += 1
    except BrokenInvariantError as e:
        print(f"ROUTE_ID={id} (BWD), DIRECTION={ris_direction};  ", e)
        stats['fail'] += 1
    except BaseException as e:
        print(f"ROUTE_ID={id} (BWD), DIRECTION={ris_direction};  ", e)
        stats['fail'] += 1
    
print('\n\nPassing: {:.2f}%'.format(100 * 10329.0 / (10329 + 205.0)))

ROUTE_ID=100514081 (FWD), DIRECTION=0;   NO PATH: id=100514081, reversed=False, u=41411743, v=2565929760
ROUTE_ID=100899011 (BWD), DIRECTION=0;   NO PATH: id=100899011, reversed=True, u=41449071, v=10655540254
ROUTE_ID=102425011 (FWD), DIRECTION=0;   NO PATH: id=102425011, reversed=False, u=41541826, v=8004928189
ROUTE_ID=272523011 (BWD), DIRECTION=0;   NO PATH: id=272523011, reversed=True, u=2451278714, v=2451278729
ROUTE_ID=272524011 (BWD), DIRECTION=0;   NO MATCH
ROUTE_ID=272527011 (BWD), DIRECTION=0;   NO PATH: id=272527011, reversed=True, u=41248640, v=2568976519
ROUTE_ID=272528011 (BWD), DIRECTION=0;   NO PATH: id=272528011, reversed=True, u=2451121229, v=2451121319
ROUTE_ID=272532011 (BWD), DIRECTION=0;   NO PATH: id=272532011, reversed=True, u=41330348, v=2078205369
ROUTE_ID=272544011 (BWD), DIRECTION=0;   NO PATH: id=272544011, reversed=True, u=41252243, v=2551057029
ROUTE_ID=272545011 (BWD), DIRECTION=0;   NO PATH: id=272545011, reversed=True, u=41287530, v=41277143
ROUTE_ID=

In [162]:
10329.0 / (10329 + 205.0)

0.9805392063793431